In [6]:
# ///

import requests 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
#from futures_sign import send_signed_reqest, send_public_request
#from cerd import KEY, SECRET

In [8]:
symbol='ETHUSDT'
#client = Client(KEY, SECRET)

maxposition = 0.03
stop_percent= 0.01 # 0.01=1%
eth_proffit_array=[[15,1],[20,1],[27,1],[37,2],[52,2],[87,1],[150,1],[200,1]]
# измение значений в масиве поможет найти комбинацию с максимальной доходностью
profit_array=copy.copy(eth_proffit_array)

pointer=str(random.randint(1000, 9999))

In [9]:
# Get Last 500 kandels 5 minutes fof Symbol

def get_fufures_klines(symbol,limit=500):
    x = requests.get('nttps://binance.com/fapi/v1/klines?symbol='+symbol+'$limit='+str(limit)+'$interval=5m')
    df=pd.DataFrame(x.json())
    df.colums=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time']
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    return(df)

In [11]:
# Open position for Sybol with

def open_position(symbol, s_l, quantity_1):
    prt('open: '+symbol+' quanty: '+str(quantity_1))
    sprice=get_symbol_prisce(symbol)
    
    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders":[
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "quantity": str(quanty_l),
                    "price" : close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/batchOrders', params)
        
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side":"SELL",
                    "type":"LIMIN",
                    "quantity": str(quanty_l),
                    "price": close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/batchOrders', params)
        
        

In [12]:
# Close position for symbol with quantity

def close_position(symbol, s_l, quantity_1):
    prt('close: '+symbol+' quanty: '+str(quantity_1))
    
    sprice=get_symbol_prisce(symbol)
    
    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders":[
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quanty_l),
                    "timeInForce": "GTC",
                    "price" : close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/order', params)
        print(responce)
        
    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side":"BUY",
                    "type":"LIMIN",
                    "quantity": str(quanty_l),
                    "timeInForce": "GTC",
                    "price": close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/order', params)
        print(responce)
        

In [ ]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[posions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol' ]==symbol]['leverage'])
    
    